## Day 4: Repose Record

https://adventofcode.com/2018/day/4

### Part 1

The chosen statistical method has its shortcomings. 

First sort the records by date and time.

In [1]:
import parse
from datetime import datetime
from collections import defaultdict, Counter, namedtuple

def parse_records(input):
    parsed = [parse.parse('[{date}] {event}', record.strip()) 
              for record in input]
    records = sorted((datetime.strptime(record['date'], '%Y-%m-%d %H:%M'), record['event']) for record in parsed)
    return records

In [2]:
test_data = '''[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up'''.splitlines()

test_records = parse_records(test_data)
test_records[:5]

[(datetime.datetime(1518, 11, 1, 0, 0), 'Guard #10 begins shift'),
 (datetime.datetime(1518, 11, 1, 0, 5), 'falls asleep'),
 (datetime.datetime(1518, 11, 1, 0, 25), 'wakes up'),
 (datetime.datetime(1518, 11, 1, 0, 30), 'falls asleep'),
 (datetime.datetime(1518, 11, 1, 0, 55), 'wakes up')]

In [3]:
input_records = parse_records(open('input', 'r'))
input_records[:5]

[(datetime.datetime(1518, 2, 3, 0, 3), 'Guard #691 begins shift'),
 (datetime.datetime(1518, 2, 3, 0, 12), 'falls asleep'),
 (datetime.datetime(1518, 2, 3, 0, 28), 'wakes up'),
 (datetime.datetime(1518, 2, 3, 0, 43), 'falls asleep'),
 (datetime.datetime(1518, 2, 3, 0, 55), 'wakes up')]

Count how often each guard is asleep for each minute of the hour.

In [4]:
def count_minutes(sorted_records):
    minutes_count = defaultdict(Counter)
    guard_on_duty = None
    fell_asleep = None
    
    for date, event in sorted_records:
        if event == "falls asleep":
            fell_asleep = date.minute
        elif event == "wakes up":
            minutes_count[guard_on_duty].update(range(fell_asleep, date.minute))
        else: 
            guard_on_duty = parse.parse('Guard #{guard:d} begins shift', event)['guard']
        
    return minutes_count

test_count = count_minutes(test_records)
assert test_count[10][24] == 2

Rigorous testing there.

This is fiddly. The sleepiest guard is the one with the highest total count of minutes asleep, and the sleepiest minute is the one the sleepiest guard most frequently slept through.

In [5]:
def answer(minutes_count):
    sleepiest_guard = max(minutes_count, 
                          key=lambda guard: sum(minutes_count[guard].values()))
    sleepiest_minute = max(minutes_count[sleepiest_guard], 
                           key=lambda minute: minutes_count[sleepiest_guard][minute])
    return sleepiest_guard * sleepiest_minute

assert answer(count_minutes(test_records)) == 240

In [6]:
input_count = count_minutes(input_records)

answer(input_count)

104764

### Part 2

Even more fiddly. Find the sleepiest minute for each guard, keeping track of how sleepy they were so we can then find the sleepiest minute among those minutes, plus keeping track of which guard it was so we can calculate the answer. This isn't very elegant.

In [7]:
RegularSleep = namedtuple("RegularSleep", "frequency guard minute")

def answer_2(minutes_count):
    sleepiest_minutes = []
    
    for guard in minutes_count:
        sleepiest_minute = max(minutes_count[guard], 
                               key=lambda minute: minutes_count[guard][minute])
        sleepiest_minutes.append(RegularSleep(minutes_count[guard][sleepiest_minute],
                                              guard,
                                              sleepiest_minute))
        
    most_regular_sleeper = max(sleepiest_minutes)
    return most_regular_sleeper.guard * most_regular_sleeper.minute

assert answer_2(count_minutes(test_records)) == 4455

In [8]:
answer_2(count_minutes(input_records))

128617